<a href="https://colab.research.google.com/github/Joshuajee/AI-ML-PROJECTS/blob/master/Topic Modelling on Financial Posts from Redit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
def get_data_from_redit(subreddit_name, num_posts, start=0):

  reddit = praw.Reddit(client_id='crOrGswYcFGg1sY48bETtw', client_secret='4FLSwEV49dFrb_q6b4f8Club3Rjxew', user_agent='MyRedditScript/1.0')

  # Define a list to store the extracted data
  data = []

  subreddit = reddit.subreddit(subreddit_name)

  # Use pagination to retrieve the posts in batches of 100
  for i in range(0, num_posts, 100):
      # Use the PRAW API to search for posts containing the keywords
      #try:
        results = subreddit.new(limit=100, params={'after': start + i})
        # Process the search results
        for result in results:
            # Check if the post has a non-empty selftext field
            if result.selftext:
                # Extract the relevant data from each post
                post_title = result.title
                post_text = result.selftext
                #post_author = result.author.name
                post_created_utc = result.created_utc
                # Store the data in the list
                data.append({
                    'title': post_title,
                    'text': post_text,
                    #'author': post_author,
                    'created_utc': post_created_utc
                  })
        # Delay for 10 seconds before making the next request
        time.sleep(10)
      #except:
        print("Request Failed")


      # Convert the list of dictionaries to a pandas DataFrame
        return pd.DataFrame(data)


In [25]:
reddit_data = get_data_from_redit('stocks', 100, 100)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Request Failed


In [27]:
reddit_data

,title,text,created_utc
0,Confused about IncomeShares with 139.10% Yield,I recently came across IncomeShares and their ...,1.742383e+09
1,Tesla gets first in a series of permits it nee...,"SAN FRANCISCO, March 18 (Reuters) - Tesla [(TS...",1.742378e+09
2,"r/Stocks Daily Discussion Wednesday - Mar 19, ...",These daily discussions run from Monday to Fri...,1.742377e+09
3,"Resources on Trading, Specifically Different S...",I’ve been wanting to dive into stock trading f...,1.742372e+09
4,Which Screener allows you to search stocks tha...,This seems so basic that I feel like I'm missi...,1.742365e+09
...,...,...,...
95,Apple’s Siri Chief Calls Siri AI Delays Ugly a...,https://www.bloomberg.com/news/articles/2025-0...,1.741978e+09
96,Europe Defense ETF,Here are all the companies included in the **W...,1.741974e+09
97,Thoughts on Target stock (NYSE: TGT)? It is at...,Bought some Target stock (NYSE: TGT) today. It...,1.741972e+09
98,"EU’s big Starlink headache is time, not money","LONDON, March 14 (Reuters Breakingviews) - As ...",1.741971e+09


In [ ]:
# Step 2: Preprocessing

def preprocess(text):
    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9\s]+', '', text)  # Remove special characters
    text = text.lower()  # Lowercase text
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text


def preprocess_dataframe(df):
    df['cleaned_text'] = df['text'].apply(preprocess)
    return df


# Step 3: Embedding Generation

model = SentenceTransformer('all-mpnet-base-v2')

def embed_text(text_list):
    return model.encode(text_list, show_progress_bar=True)


# Step 4: Topic Modeling (BERTopic)

def create_topics(texts):
    embeddings = embed_text(texts)
    topic_model = BERTopic()
    topics, probs = topic_model.fit_transform(texts, embeddings)
    return topic_model, topics, probs


# Step 5: Fetching Data for Both Subreddits
stock_data = fetch_reddit_posts('stocks', num_posts=500)
ai_data = fetch_reddit_posts('artificialintelligence', num_posts=500)

# Step 6: Preprocessing Data
stock_data = preprocess_dataframe(stock_data)
ai_data = preprocess_dataframe(ai_data)

# Step 7: Creating Topics
stock_model, stock_topics, stock_probs = create_topics(stock_data['cleaned_text'].tolist())
ai_model, ai_topics, ai_probs = create_topics(ai_data['cleaned_text'].tolist())

# Step 8: Visualize Topics
stock_model.visualize_topics().show()
ai_model.visualize_topics().show()


In [26]:
get_data_from_redit('stocks', 100, 100)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Request Failed


,title,text,created_utc
0,Confused about IncomeShares with 139.10% Yield,I recently came across IncomeShares and their ...,1.742383e+09
1,Tesla gets first in a series of permits it nee...,"SAN FRANCISCO, March 18 (Reuters) - Tesla [(TS...",1.742378e+09
2,"r/Stocks Daily Discussion Wednesday - Mar 19, ...",These daily discussions run from Monday to Fri...,1.742377e+09
3,"Resources on Trading, Specifically Different S...",I’ve been wanting to dive into stock trading f...,1.742372e+09
4,Which Screener allows you to search stocks tha...,This seems so basic that I feel like I'm missi...,1.742365e+09
...,...,...,...
95,Apple’s Siri Chief Calls Siri AI Delays Ugly a...,https://www.bloomberg.com/news/articles/2025-0...,1.741978e+09
96,Europe Defense ETF,Here are all the companies included in the **W...,1.741974e+09
97,Thoughts on Target stock (NYSE: TGT)? It is at...,Bought some Target stock (NYSE: TGT) today. It...,1.741972e+09
98,"EU’s big Starlink headache is time, not money","LONDON, March 14 (Reuters Breakingviews) - As ...",1.741971e+09
